### instalar dependencias

In [76]:
# ! pip install pandas
# ! pip install requests
# ! pip install beautifulsoup4
# ! pip install matplotlib
# ! pip install mlxtend

### Obtener data

In [77]:
import requests
import json

try:
    with open("jobs.json", 'r', encoding='utf-8') as file:
        jobs = json.load(file)

except FileNotFoundError:
    response = requests.get("https://www.getonbrd.com/api/v0/search/jobs", params={
        "query": "Software engineering",
        "per_page": 100,
        "page": 1,
    })

    jobs = response.json()["data"]

    with open("jobs.json", 'w', encoding='utf-8') as file:
        json.dump(jobs, file, ensure_ascii=False, indent=4)
        



### guardar data

In [78]:
import json

def guardar_en_json(array_de_objetos, nombre_archivo):
    
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo_json:
        json.dump(array_de_objetos, archivo_json, ensure_ascii=False, indent=4)


guardar_en_json(jobs, 'jobs.json')

### Construir data frame

In [79]:
import pandas as pd
from bs4 import BeautifulSoup

def transform_jobs_to_dataframe(jobs):
    
    structured_data = []
    for job in jobs:
        job_data = {
            "id": job["id"],
            "title": job["attributes"]["title"],
            "description": BeautifulSoup(job["attributes"].get("description", ""), 'html.parser').get_text(),
            "projects": BeautifulSoup(job["attributes"].get("projects", ""), 'html.parser').get_text(),
            "functions": BeautifulSoup(job["attributes"].get("functions", ""), 'html.parser').get_text(),
            "benefits": BeautifulSoup(job["attributes"].get("benefits", ""), 'html.parser').get_text(),
            "desirable": BeautifulSoup(job["attributes"].get("desirable", ""), 'html.parser').get_text(),
            "seniority": job["attributes"].get("seniority", ""),
            "remote": job["attributes"]["remote"],
            "remote_modality": job["attributes"].get("remote_modality", ""),
            "remote_zone": job["attributes"].get("remote_zone", ""),
            "countries": job["attributes"]["countries"],
            "lang": job["attributes"]["lang"],
            "category_name": job["attributes"]["category_name"],
            "perks": job["attributes"]["perks"],
            "min_salary": job["attributes"].get("min_salary"),
            "max_salary": job["attributes"].get("max_salary"),
            "published_at": job["attributes"]["published_at"],
            "response_time_min": job["attributes"]["response_time_in_days"].get("min"),
            "response_time_max": job["attributes"]["response_time_in_days"].get("max"),
            "applications_count": job["attributes"]["applications_count"],
            "public_url": job["links"]["public_url"]
        }
        structured_data.append(job_data)
    
    return pd.DataFrame(structured_data)

df_jobs = transform_jobs_to_dataframe(jobs)
df_jobs

def get_seniority_type(seniority):
    return seniority['data']['type']

df_jobs['seniority_type'] = df_jobs['seniority'].apply(get_seniority_type)

seniority_index = df_jobs.columns.get_loc('seniority')

df_jobs.insert(seniority_index + 1, 'seniority_type', df_jobs.pop('seniority_type'))

df_jobs



,id,title,description,projects,functions,benefits,desirable,seniority,seniority_type,remote,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
0,android-engineer-haystack-news-lima-3ad4,Android Engineer,Strong written and spoken English is a must!B....,Haystack News is the leading local & world new...,You'll be responsible for developing the Hayst...,Unlimited vacations :)Travel to team's offsite...,,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,False,...,en,Mobile Development,"[accessible, relocation, pet_friendly, flexibl...",NaN,NaN,1717080915,NaN,NaN,93,https://www.getonbrd.com/jobs/android-engineer...
1,senior-qa-qc-automation-softserve-remote,Senior QA/QC (Manual),Possessing a Bachelor's degree in Computer Sci...,WE ARESoftServe is a IT & global digital solut...,Collaborate closely with software engineers re...,Gain certifications from leading providers (Go...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,True,...,en,SysAdmin / DevOps / QA,"[pet_friendly, flexible_hours, health_coverage...",NaN,NaN,1717000763,NaN,NaN,41,https://www.getonbrd.com/jobs/senior-qa-qc-aut...
2,machine-learning-engineer-neuralworks-santiago...,Machine Learning Engineer,Ingeniería Civil en Computación o similarAl me...,NeuralWorks es una compañía de alto crecimient...,El equipo de analytics trabaja en diferentes p...,MacBook Air M1 o similar (con opción de compra...,"Experiencia en Infraestructura as code, observ...","{'data': {'id': 3, 'type': 'seniority'}}",seniority,False,...,es,Machine Learning & AI,"[library, accessible, flexible_hours, internal...",NaN,NaN,1716241209,NaN,NaN,86,https://www.getonbrd.com/jobs/machine-learning...
3,software-engineer-flutter-humanforest-santiago,Software Engineer Flutter,Essential Experience:2+ years of experience as...,HumanForest is a forward-thinking company that...,Join the development of Forest’s mobile applic...,,Knowledge of a state management system (BLOC d...,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,False,...,en,Mobile Development,"[pet_friendly, flexible_hours, remote_partial,...",3500.0,5100.0,1715886739,NaN,NaN,48,https://www.getonbrd.com/jobs/software-enginee...
4,solutions-engineer-openloop-remote,Solutions Engineer,3+ years experience building production-ready ...,OpenLoop is looking for a Solutions Engineer t...,Take complex screen designs and quickly render...,"In addition to competitive salaries, this role...",Experience working with Zoho and Healthie is c...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,False,...,en,Programming,"[life_insurance, paid_sick_days, bicycle_parki...",NaN,NaN,1715884417,11.0,21.0,79,https://www.getonbrd.com/jobs/solutions-engine...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,junior-ios-software-engineer-kti-hunter-remote,Junior iOS Software Engineer,ExperienciaExperienced for 2+ years in program...,"Somos una Empresa de Tecnología innovadora, co...","DeberesDevelop scalable, robust, fault-toleran...",BeneficiosSeguro ComplementarioProgramas de Bi...,,"{'data': {'id': 2, 'type': 'seniority'}}",seniority,True,...,lang_not_specified,Mobile Development,"[flexible_hours, health_coverage, computer_pro...",1400.0,2100.0,1707345234,NaN,NaN,85,https://www.getonbrd.com/jobs/junior-ios-softw...
66,software-engineer-full-stack-apply-digital-lat...,Software Engineer (Full-Stack),Strong working knowledge of JavaScript ES6 and...,If you'd like to learn more about Apply Digita...,We are seeking a dedicated and innovative Full...,Flexibility - work where you work bestCompetit...,AWSPostgreSQLKubernetesGraphQLJAMstackNetlifyC...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,True,...,lang_not_specified,Programming,"[remote_full, flexible_hours, health_coverage,...",NaN,NaN,1704808682,NaN,NaN,582,https://www.getonbrd.com/jobs/software-enginee...
67,intermediate-qa-automation-engineer-apply-digi...,Intermediate QA Automa

In [80]:
# relacion entre las perks
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

jobs_perks = df_jobs['perks'].tolist()

te = TransactionEncoder()
te_ary = te.fit_transform(jobs_perks)

df = pd.DataFrame(te_ary, columns=te.columns_)
df

frequent_perks = apriori(df, min_support=0.1, use_colnames=True)

print("Ítems frecuentes encontrados:")
frequent_perks

rules = association_rules(frequent_perks, metric="confidence", min_threshold=0.5)

rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

rulesRemote = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rulesRemote

Ítems frecuentes encontrados:


,antecedents,consequents,support,confidence,lift
0,bicycle_parking,accessible,0.114286,0.571429,5.000000
1,accessible,bicycle_parking,0.114286,1.000000,5.000000
2,accessible,computer_provided,0.114286,1.000000,1.250000
3,accessible,flexible_hours,0.100000,0.875000,1.361111
4,accessible,informal_dresscode,0.114286,1.000000,1.320755
...,...,...,...,...,...
3883,"beverages_and_snacks, health_coverage","vacation_over_legal, informal_dresscode, pet_f...",0.128571,0.529412,2.850679
3884,"pet_friendly, flexible_hours","beverages_and_snacks, vacation_over_legal, inf...",0.128571,0.500000,2.058824
3885,"vacation_over_legal, pet_friendly","beverages_and_snacks, flexible_hours, informal...",0.128571,0.600000,3.230769
3886,"computer_provided, pet_friendly","beverages_and_snacks, flexible_hours, vacation...",0.128571,0.500000,2.692308


In [81]:
# relacion entre las remote y remote modality
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
parsed = df_jobs

parsed['remote'] = df_jobs['remote'].replace({True: 'remote', False: 'no_remote'})
parsed
remote_array = parsed[['remote', 'remote_modality']].values.tolist()
remote_array
te = TransactionEncoder()
te_re = te.fit_transform(remote_array)
te_re
df = pd.DataFrame(te_re, columns=te.columns_)

frequent_remote = apriori(df, min_support=0.1, use_colnames=True)

print("Ítems frecuentes encontrados:")
frequent_remote

rulesRemote = association_rules(frequent_remote, metric="confidence", min_threshold=0.5)

# Convertir los conjuntos en strings y eliminar los paréntesis
rulesRemote['antecedents'] = rulesRemote['antecedents'].apply(lambda x: ', '.join(list(x)))
rulesRemote['consequents'] = rulesRemote['consequents'].apply(lambda x: ', '.join(list(x)))

rulesRemote = rulesRemote[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
print(rulesRemote)

Ítems frecuentes encontrados:
Reglas de asociación generadas:
    antecedents   consequents   support  confidence      lift
0  fully_remote        remote  0.271429       1.000  1.750000
1     no_remote        hybrid  0.385714       0.900  2.333333
2        hybrid     no_remote  0.385714       1.000  2.333333
3  remote_local        remote  0.300000       1.000  1.750000
4        remote  remote_local  0.300000       0.525  1.750000


In [92]:
# relacion entre las country y category
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

df_jobs
array_category_country = []
for index, row in df_jobs.iterrows():
    category_name = row['category_name']
    countries = row['countries']

    array_category_country.append([category_name] + countries)

te = TransactionEncoder()
te_co = te.fit_transform(array_category_country)

df = pd.DataFrame(te_co, columns=te.columns_)
df

frequent_country_category = apriori(df, min_support=0.1, use_colnames=True)
frequent_country_category
rules_country_category = association_rules(frequent_country_category, metric="confidence", min_threshold=0.5)

# Convertir los conjuntos en strings y eliminar los paréntesis
rules_country_category['antecedents'] = rules_country_category['antecedents'].apply(lambda x: ', '.join(list(x)))
rules_country_category['consequents'] = rules_country_category['consequents'].apply(lambda x: ', '.join(list(x)))

rules_country_category = rules_country_category[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
print("Reglas de asociación generadas:")
rules_country_category


Reglas de asociación generadas:


,antecedents,consequents,support,confidence,lift
0,Chile,Programming,0.142857,0.555556,1.051051
1,SysAdmin / DevOps / QA,Remote,0.242857,0.850000,1.487500


In [83]:
# 1. ¿Qué porcentaje de trabajos requieren experiencia en 'Remote' o 'Hybrid' en comparación con los trabajos presenciales?
#Enfoque: Análisis de proporciones. Se calculará la proporción de trabajos remotos, híbridos y para obtener porcentajes exactos.

cantidad = df_jobs["remote"].value_counts()
porcentaje = df_jobs["remote"].value_counts(normalize=True)
pd.concat([cantidad,porcentaje], axis=1, keys=['cantidad', 'porcentaje'])

,cantidad,porcentaje
remote,,
remote,40,0.571429
no_remote,30,0.428571


In [84]:
# 2. ¿Cuáles son las tres habilidades más comúnmente solicitadas en la categoría de 'Data Science / Analytics'?
    
df_categories_data_anaytics = df_jobs[df_jobs['category_name'] == 'Data Science / Analytics']

all_desirable_skills = df_categories_data_anaytics['desirable'].str.split(', ').explode()

skills_count = all_desirable_skills.value_counts()

print("Frecuencia de habilidades en la categoría 'Data Science / Analytics':")
skills_count

# Filtro por categoria "Data Science/ Analytics". Se debe analizar la informacion manualmente ya que el texto no es categorico.

Frecuencia de habilidades en la categoría 'Data Science / Analytics':


desirable
                                                                                                   1
Experiencia en Infraestructura as code                                                             1
observabilidad                                                                                     1
monitoreoExperiencia en la construcción y optimización de data pipelines                           1
colas de mensajes y arquitecturas big data altamente escalablesExperiencia en Stream processing    1
Name: count, dtype: int64

In [85]:
#3. ¿Cuáles trabajos requieren certificaciones específicas (como PL-300 para Power BI)?

descripciones_con_certificacion = df_jobs[df_jobs['description'].str.contains('certificación|certificacion', case=False, na=False)]
descripciones_con_certificacion
# El enfoque es el siguiente, se filtran cuando en la columna descripcion aparece la palafra certificacion, esto permite obtener las ofertas que requieren certificacion

,id,title,description,projects,functions,benefits,desirable,seniority,seniority_type,remote,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
18,especialista-en-administracion-de-infraestruc-...,Especialista en Administración de Infraestruc,✅Estamos buscando un Especialista en Administr...,En Lilab atendemos clientes locales y del extr...,En esta oportunidad nos encontramos buscando u...,¡Únete y desarrolla tu potencial con nosotros!...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,SysAdmin / DevOps / QA,"[wellness, flexible_hours, internal_talks, hea...",1400.0,1900.0,1716402868,NaN,NaN,28,https://www.getonbrd.com/jobs/especialista-en-...
24,engineer-manager-iconstruye-remote-64d6,Engineer Manager,"Más de 5 años de experiencia en TI, con al men...",¡Súmate a nuestro equipo! Estamos buscando a n...,Ser responsable de supervisar y guiar a nuestr...,5 días extras de descanso (según nuestra polít...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,no_remote,...,lang_not_specified,Programming,"[flexible_hours, computer_provided, informal_d...",NaN,NaN,1715181940,NaN,NaN,27,https://www.getonbrd.com/jobs/engineer-manager...
32,lider-qa-tester-migracion-flexcube-sophos-solu...,Lider QA Tester Migración Flexcube,2 a 3 años de experiencia mínima como: Analist...,Sophos Solutions es una compañía global de so...,Participar en el levantamiento de requerimient...,Ingreso: 7.5 millones COP al mes Horario: de ...,Manejo de algún repositorio como Github o Bitb...,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,SysAdmin / DevOps / QA,"[wellness, health_coverage, mobile_provided, c...",NaN,NaN,1715106105,NaN,NaN,162,https://www.getonbrd.com/jobs/lider-qa-tester-...


In [86]:
# 4. ¿Qué porcentaje de trabajos requiere un nivel avanzado de inglés?
#Calcular los pocentajes de cada idioma requerido en las ofertas de trabajo. Luego obtener la categoría específica de nivel avanzado.


cantidad = df_jobs["lang"].value_counts()
porcentaje = df_jobs["lang"].value_counts(normalize=True)
pd.concat([cantidad,porcentaje], axis=1, keys=['cantidad', 'porcentaje'])

,cantidad,porcentaje
lang,,
en,31,0.442857
lang_not_specified,25,0.357143
es,14,0.200000


In [87]:
# 5. ¿Cuántos trabajos requieren experiencia previa específica en la industria (por ejemplo, agrícola, ciberseguridad)?
#Enfoque: analizar cantidad de trabajos que requieren experiencia

cantidad = df_jobs["seniority"].value_counts()
print(cantidad)
porcentaje = df_jobs["seniority"].value_counts(normalize=True)
pd.concat([cantidad,porcentaje], axis=1, keys=['cantidad', 'porcentaje'])

seniority
{'data': {'id': 4, 'type': 'seniority'}}    38
{'data': {'id': 3, 'type': 'seniority'}}    22
{'data': {'id': 2, 'type': 'seniority'}}     7
{'data': {'id': 5, 'type': 'seniority'}}     3
Name: count, dtype: int64


,cantidad,porcentaje
seniority,,
"{'data': {'id': 4, 'type': 'seniority'}}",38,0.542857
"{'data': {'id': 3, 'type': 'seniority'}}",22,0.314286
"{'data': {'id': 2, 'type': 'seniority'}}",7,0.100000
"{'data': {'id': 5, 'type': 'seniority'}}",3,0.042857


In [88]:
#6. ¿Cuáles trabajos en 'Design / UX' requieren experiencia en herramientas específicas como Figma o Adobe Illustrator?
df_categories_design_ux = df_jobs[df_jobs['category_name'] == 'Design / UX']

df_figma_adobe = df_categories_design_ux[
    df_categories_design_ux['description'].str.contains('Figma|Adobe Illustrator', case=False, na=False)
]
len(df_figma_adobe)

# El enfoque es el siguiente, se filtran por categoria Design / UX y luego se filtran los que en la columna descripcion tengan la palabra Figma u Adobe Ilustrator.

1

In [89]:
# 8. ¿Cuántos trabajos ofrecen beneficios relacionados con la salud (como seguro de salud o dental)?

df_health_perks = df_jobs[
    df_jobs['perks'].apply(lambda x: any('health' in s for s in x)) |
    df_jobs['benefits'].str.contains('health', case=False, na=False)
]
df_health_perks
# El enfoque tomado es obtener los datos donde en la columna perks o benefith este presente la palabra health.


,id,title,description,projects,functions,benefits,desirable,seniority,seniority_type,remote,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
1,senior-qa-qc-automation-softserve-remote,Senior QA/QC (Manual),Possessing a Bachelor's degree in Computer Sci...,WE ARESoftServe is a IT & global digital solut...,Collaborate closely with software engineers re...,Gain certifications from leading providers (Go...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,remote,...,en,SysAdmin / DevOps / QA,"[pet_friendly, flexible_hours, health_coverage...",NaN,NaN,1717000763,NaN,NaN,41,https://www.getonbrd.com/jobs/senior-qa-qc-aut...
3,software-engineer-flutter-humanforest-santiago,Software Engineer Flutter,Essential Experience:2+ years of experience as...,HumanForest is a forward-thinking company that...,Join the development of Forest’s mobile applic...,,Knowledge of a state management system (BLOC d...,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,no_remote,...,en,Mobile Development,"[pet_friendly, flexible_hours, remote_partial,...",3500.0,5100.0,1715886739,NaN,NaN,48,https://www.getonbrd.com/jobs/software-enginee...
4,solutions-engineer-openloop-remote,Solutions Engineer,3+ years experience building production-ready ...,OpenLoop is looking for a Solutions Engineer t...,Take complex screen designs and quickly render...,"In addition to competitive salaries, this role...",Experience working with Zoho and Healthie is c...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,no_remote,...,en,Programming,"[life_insurance, paid_sick_days, bicycle_parki...",NaN,NaN,1715884417,11.0,21.0,79,https://www.getonbrd.com/jobs/solutions-engine...
6,full-stack-angular-java-lilab-ica-13a2,Full-Stack Angular + Java,✅Estamos buscando un Full Stack Senior con mín...,En Lilab atendemos clientes locales y del extr...,En esta oportunidad nos encontramos buscando u...,Únete y desarrolla tu potencial con nosotros!T...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,no_remote,...,lang_not_specified,Programming,"[wellness, life_insurance, health_coverage, in...",1800.0,2100.0,1716936602,NaN,NaN,4,https://www.getonbrd.com/jobs/full-stack-angul...
8,senior-drupal-engineer-devsu-remote-92fa,Semi Senior/Senior Rust Developer,Experience: At least 4 years of professional e...,With over 10 years of experience creating inno...,"Architect, design, and implement software solu...","A stable, long-term contract. Continuous Train...",,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,Programming,"[wellness, remote_full, internal_talks, paid_s...",NaN,NaN,1716820720,NaN,NaN,24,https://www.getonbrd.com/jobs/senior-drupal-en...
12,desarollador-fullstack-iconstruye-remote-c104,Desarrollador Full-Stack,"Lenguaje de Programación Angular, JavaScript, ...",¡Súmate a nuestro equipo! Estamos buscando a n...,Colaborar en definiciones arquitectónicas TI.E...,5 días extras de descanso.Tarjeta amipass para...,,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,no_remote,...,es,Programming,"[flexible_hours, health_coverage, computer_pro...",NaN,NaN,1716927867,NaN,NaN,129,https://www.getonbrd.com/jobs/desarollador-ful...
13,cloud-devops-technical-manager-devsu-remote,Cloud & DevOps Technical Manager,Hands-on experience with the technologies you’...,With over 10 years of experience creating inno...,Cloud and DevOps Team Leadership:Lead and mana...,"A stable, long-term contract. Continuous Train...",Experience with containerization technologies ...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,SysAdmin / DevOps / QA,"[wellness, remote_full, equity_offered, intern...",NaN,NaN,1716823042,NaN,NaN,74,https://www.getonbrd.com/jobs/cloud-devops-tec...
14,qa-automation-engineer-sector-financiero-devsu...,QA Automation Engineer Sector Financiero,Orientado al detalle.Habilidades de programaci...,Con más de 10 años de experiencia creando inn

In [90]:
# 9. ¿Cuántos trabajos se publicaron en el último mes?
from datetime import datetime

mes_actual = datetime.now().month

df_mes_actual = df_jobs[df_jobs['published_at'].apply(lambda x: datetime.fromtimestamp(x).month == mes_actual)]

len(df_mes_actual)

# nos centramos en obtener las filas en las cuales el mes sea equivalente al mes actual, de esa manera podemos obtener el numero trabajos de este mes.



37

In [91]:
# 10. ¿Cuáles trabajos tienen la mayor cantidad de aplicaciones hasta la fecha?
df_ordenas = df_jobs.sort_values(by='applications_count', ascending=False)

df_ordenas

# enfoque, se busco ordenar los trabajos de mayor a menor segun el numero de aplicaciones, de esta forma podemos obtener ordenadamente que trabajos tienen mas aplicaciones.


,id,title,description,projects,functions,benefits,desirable,seniority,seniority_type,remote,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
50,back-end-software-engineer-jr-foris-ai-remote-...,Back-end Software Engineer Jr,Tienes un nivel junior en desarrollo Back-end:...,Desarrollamos una suite de soluciones cloud ba...,"En Foris, buscamos desarrolladores proactivos,...","Somos un equipo multidisciplinario, motivado, ...",Soluciones cloud de AWS para despliegue de apl...,"{'data': {'id': 2, 'type': 'seniority'}}",seniority,remote,...,es,Programming,"[flexible_hours, internal_talks, company_retre...",1500.0,2300.0,1712354944,0.0,6.0,1425,https://www.getonbrd.com/jobs/back-end-softwar...
11,fullstack-developer-rankmi-remote-82fc,RoR React Developer,Conocimiento y experiencia de al menos 2 años ...,"💖 ¿Por qué trabajar en Rankmi?En Rankmi, no só...",Desarrollo de funcionalidades Back-end en RoR....,🏄🏼‍♀️ Work-Life BalanceBlended Working: Nuest...,,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,remote,...,es,Programming,"[remote_full, pet_friendly, flexible_hours, in...",NaN,NaN,1717015603,13.0,17.0,678,https://www.getonbrd.com/jobs/fullstack-develo...
66,software-engineer-full-stack-apply-digital-lat...,Software Engineer (Full-Stack),Strong working knowledge of JavaScript ES6 and...,If you'd like to learn more about Apply Digita...,We are seeking a dedicated and innovative Full...,Flexibility - work where you work bestCompetit...,AWSPostgreSQLKubernetesGraphQLJAMstackNetlifyC...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,Programming,"[remote_full, flexible_hours, health_coverage,...",NaN,NaN,1704808682,NaN,NaN,582,https://www.getonbrd.com/jobs/software-enginee...
54,qa-automation-engineer-outbuild-remote,QA Automation Engineer,Technical RequirementsBachelor’s degree in Com...,Outbuild es una startup con sus orígenes en La...,Outbuild is looking for a QA Automation Engine...,In our team you will also have100% remote work...,Strong experience developing automation tests ...,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,remote,...,lang_not_specified,SysAdmin / DevOps / QA,"[remote_full, flexible_hours, computer_provide...",2000.0,3000.0,1711988523,NaN,NaN,544,https://www.getonbrd.com/jobs/qa-automation-en...
58,qa-engineer-tongo-remote,QA Engineer,Qualifications:· A minimum of 3 years as a QA ...,Financial products are built for salaried prof...,Responsibilities:· Execute test cases and scen...,,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,remote,...,en,SysAdmin / DevOps / QA,[remote_full],3000.0,5000.0,1710878496,NaN,NaN,402,https://www.getonbrd.com/jobs/qa-engineer-tong...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,lead-engineer-hybrid-stori-remote,Lead Engineer (Hybrid),Experience managing teams and designing soluti...,"Stori is a fast-growing, venture-backed financ...",Stori is looking for candidates with experienc...,Make a positive impact on the lives of our cus...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,no_remote,...,en,Programming,"[wellness, pet_friendly, flexible_hours, meals...",NaN,NaN,1709925173,NaN,NaN,8,https://www.getonbrd.com/jobs/lead-engineer-hy...
53,senior-devsecops-engineer-zerofox-santiago,Senior DevSecOps Engineer,Strong understanding of the software developme...,"At ZeroFox, we are passionate about bringing w...",Work closely with the Security product owner t...,Opportunities to learn and contribute your kno...,Exceptional scripting skills. Experience with ...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,no_remote,...,en,Cybersecurity,"[life_insurance, meals_provided, remote_partia...",NaN,NaN,1712082865,NaN,NaN,5,https://www.getonbrd.com/jobs/senior-devsecops...
6,full-stack-angular-java-lilab-ica-13a2,Full-Stack Angular + Java,✅Estamos buscando un Full Stack Senior con mín...,En Lilab atendemos 